In [1]:
from flask import Flask, flash, render_template, jsonify, request, redirect, url_for, Response, session
import time
import random
import string
import logging
import logging.handlers
import sqlite3
import yfinance as yf
import plotly.offline as po
import plotly.graph_objs as go
import plotly.express as px
import json
import plotly
import pandas as pd
from functools import wraps

# template_dir = os.path.abspath('/templates')
app = Flask(__name__)
app.secret_key = 'your_secret_key'

log = logging.getLogger('flask')
log.setLevel('INFO')
h = logging.handlers.TimedRotatingFileHandler(
    'logs/nlog', when='midnight', backupCount=9999)
f = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
h.setFormatter(f)
log.addHandler(h)

def get_asset_data():
    conn = sqlite3.connect('DataBase1.db')
    c = conn.cursor()
    c.execute("SELECT assetName FROM assets")
    asset_names = [row[0] for row in c.fetchall()]
    conn.close()
    return(asset_names)

def get_full_asset_data():
    conn = sqlite3.connect('DataBase1.db')
    c = conn.cursor()
    c.execute("SELECT assetName, assetFullName FROM assets")
    asset_names = [{"assetName":name,"assetFullName":fullname} for name,fullname in c.fetchall()]
    c.fetchall()
    print(c.fetchall())
    conn.close()
    return(asset_names)

def get_stock_markets():
    conn = sqlite3.connect('DataBase1.db')
    c = conn.cursor()
    c.execute("SELECT stockSymbol FROM stockMarket")
    stock_Markets = [row[0] for row in c.fetchall()]
    conn.close()
    return(stock_Markets)


@app.route('/')
@app.route('/index/')
def index():
    title = 'Portfolio Tracker'
    return render_template('index.html', title=title)

@app.route('/register_page')
def register1():
    title = 'Register Page'
    return render_template('register1.html', title=title)

def CheckIfCorrectCredentials(loginName,password):

    conn = sqlite3.connect('DataBase1.db')

    c = conn.cursor()

    params=[loginName,password]
    c.execute("Select * from users where loginName==? and password==?",params)
    print("\nPresent all data in student table:")

    #Get all results,assign them to the list,fecthall() returns empty list if no results
    listOfResults=c.fetchall()

    # Save (commit) the changes
    conn.commit()

    # We close the connection and free all resources
    conn.close()
    
    # If given loginName and password exist in database then exactly one row should be returned.
    # We assume that loginNames are unique.
    if (len(listOfResults)==1):
        return True
    return False

@app.route('/login_page')
def login_page():
    title = 'Login Page'
    return render_template('login.html', title=title)

@app.route('/login/', methods=['POST','GET'])
def login():
    loginName = request.form.get('login_name', default=False)
    password = request.form.get('password', default=False)

    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))

    if (CheckIfCorrectCredentials(loginName,password)==True):
        session['loginName'] = loginName            # stores the loginName of logged User
        session['userID'] = takeUserID(loginName)  
        return redirect('/portfolioOverview')
    else:
        title = 'Login Page'
        return render_template('login.html', error = True, title=title)

def takeUserID(loginName):
    conn = sqlite3.connect('DataBase1.db')

    c = conn.cursor()

    c.execute("select userID from users where loginName = ?", (loginName,))

    userID = c.fetchone()[0]

    conn.close()

    return userID


@app.route('/register', methods=['POST','GET'])
def register():
    loginName = request.form.get('login_name', default=False)
    password1 = request.form.get('password1', default=False)
    password2 = request.form.get('password2', default=False)

    name = request.form.get('name', default=False)
    surname = request.form.get('surname', default=False)
    birthDate = request.form.get('birthDate', default=False)

    city = request.form.get('city', default=False)
    zipCode = request.form.get('zipCode', default=False)
    street = request.form.get('street', default=False)
    houseNo = request.form.get('houseNo', default=False)
    flatNo =request.form.get('flatNo', default=False)

    conn = sqlite3.connect('DataBase1.db')

    c = conn.cursor()

    c.execute("select userID from users order by userID desc limit 1")
    userID = c.fetchall()
    userID = userID[0][0]
    userID += 1
    
    c.execute("select personalInfoID from userPersonalInfo order by PersonalInfoID desc limit 1")
    personalInfoID = c.fetchall()
    personalInfoID = personalInfoID[0][0]
    personalInfoID += 1

    params1 = [loginName, password1]
    c.execute("insert into users(loginName, password) values (?, ?)", params1)

    params2 = [name, surname, birthDate, userID]
    c.execute("insert into userPersonalInfo(name, surname, birthDate, userID) values (?, ?, ?, ?)", params2)

    params3 = [city, zipCode, street, houseNo, flatNo, personalInfoID]
    c.execute("insert into userAddress(city, zipCode, street, houseNo, flatNo, personalInfoID) values (?, ?, ?, ?, ?, ?)", params3)

    conn.commit()
    conn.close()

    title = 'Portfolio Overview'

    return render_template('portfolioOverview.html', title=title)


def takeAllUserData():

    loginName = session['loginName']

    conn = sqlite3.connect('DataBase1.db')
    c = conn.cursor()
    c.execute("SELECT userID FROM users WHERE loginName = ?", (loginName,))
    
    userID = session['userID']
    
    c.execute("select * from userPersonalInfo where userID = ?", (userID,))

    userPersonalInfo = c.fetchall()[0]
    personalInfoID = userPersonalInfo[0]

    c.execute("select * from userAddress where personalInfoID = ?", (personalInfoID,))
    userAddress = c.fetchall()[0]
    
    conn.commit()
    conn.close()
    
    name = userPersonalInfo[1]
    surname = userPersonalInfo[2]
    birthDate = userPersonalInfo[3]

    city = userAddress[1]
    zipCode = userAddress[2]
    street = userAddress[3]
    houseNo = userAddress[4]
    flatNo = userAddress[5]

    return loginName, name, surname, birthDate, city, zipCode, street, houseNo, flatNo

def login_required(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        if session.get('loginName') is None:
            return redirect('/login_page')
        return f(*args, **kwargs)
    return decorated_function


@app.route('/stockMarketInfo')
@login_required
def stockMarketInfo():
    title = 'Stock Market Info'
    stock_Markets = get_stock_markets()
    asset_names = get_full_asset_data()
    return render_template('stockMarketInfo.html', title=title, asset_names = asset_names, stock_Markets = stock_Markets)

@app.route('/portfolioEditor', methods=['POST','GET'])
@login_required
def portfolioEditor():
    #conn.close()
    stock_Markets = get_stock_markets()
    asset_names = get_full_asset_data()
    title = "Portfolio Editor"
    return render_template('portfolioEditor.html', title=title, asset_names = asset_names, stock_Markets = stock_Markets)

@app.route('/addTransactionHistory', methods=['POST','GET'])
def addTransactionHistory():

    Stock_Market = request.form.get('stock_market', default=False)
    Asset = request.form.get('Asset', default=False)
    Quantity = int(request.form.get('Quantity', default=False))
    Price = request.form.get('Price', default=False)
    transactionType = request.form.get('transactionType', default=False)
    Date = request.form.get('Date', default=False)
    user_id = takeUserID(session['loginName'])  
    asset_names = get_asset_data()
    
    conn = sqlite3.connect('DataBase1.db')
    c = conn.cursor()
    Stock_Market = c.execute("select stockID from stockMarket where stockSymbol ==?", (Stock_Market,)).fetchone()[0]
    AssetID = c.execute("select assetID from assets where assetName ==?", (Asset,)).fetchone()[0]
    AssetName = c.execute("select assetFullName from assets where assetID ==?", (AssetID,)).fetchone()[0]
    conn.commit()
    conn.close()
    flash(f"You have succesfully {transactionType} {Quantity} {AssetName} stocks for the price of {Price}$")
    
    if transactionType == "Sell":
        Quantity = -Quantity

    conn = sqlite3.connect('DataBase1.db')
    c = conn.cursor()
    

    params = [user_id, Stock_Market, AssetID, Quantity, Price, transactionType,  Date]
    c.execute("insert into transactionHistory(userID,assetID,stockID,quantity,transactionPrice, transactionType, transactionDate)values (?, ?, ?, ?, ?, ?, ?)", params)

    conn.commit()
    conn.close()
    title = "Portfolio Editor"
    return render_template('portfolioEditor.html', title=title)



@app.route('/personalData')
@login_required
def personalData():
    data = takeAllUserData()        #  argument after ** must be a mapping, not tuple"
    data_dict = {
        'loginName': data[0],
        'name': data[1],
        'surname': data[2],
        'birthDate': data[3],
        'city': data[4],
        'zipCode': data[5],
        'street': data[6],
        'houseNo': data[7],
        'flatNo': data[8]
    }
    title = 'Personal Data'
    return render_template('personalData.html', title=title, **data_dict)

@app.route('/changePersonalDataPage')
@login_required
def get_data():
    title = "Change Personal Data"
    return render_template('changePersonalData.html', title=title)


@app.route('/changePersonalData', methods=['POST','GET'])
def changePersonalData():
    newLoginName = request.form.get('new_login_name', default=False)

    newName = request.form.get('new_name', default=False)
    newSurname = request.form.get('new_surname', default=False)
    newBirthDate = request.form.get('new_birthDate', default=False)

    newCity = request.form.get('new_city', default=False)
    newZipCode = request.form.get('new_zipCode', default=False)
    newStreet = request.form.get('new_street', default=False)
    newHouseNo = request.form.get('new_houseNo', default=False)
    newFlatNo =request.form.get('new_flatNo', default=False)

    conn = sqlite3.connect('DataBase1.db')

    c = conn.cursor()

    loginName = session['loginName']
    

    userID = session["userID"]

    c.execute("select personalInfoID from userPersonalInfo where userID = ?", (userID,))
    personalInfoID = c.fetchone()[0]

    c.execute("update users set loginName = ? where userID = ?", (newLoginName, userID))

    c.execute("update userPersonalInfo set name = ?, surname = ?, birthDate = ? where userID = ?", (newName, newSurname, newBirthDate, userID))

    c.execute("update userAddress set city = ?, zipCode = ?, street = ?, houseNo = ?, flatNo = ? where personalInfoID = ?", (newCity, newZipCode, newStreet, newHouseNo, newFlatNo, personalInfoID))

    conn.commit()
    conn.close()

    title = 'Change Personal Data'

    return render_template('personalData.html', title=title)

@app.route('/callback/<endpoint>', methods= ['GET', 'POST'])
def cb(endpoint):  
    if endpoint == "getStock":
        return gm(request.args.get('data'),request.args.get('period'),request.args.get('interval'))
 
    elif endpoint == "getStockMarkets":

        stock_market = request.args.get("stockMarket")
        conn = sqlite3.connect('DataBase1.db')
        c = conn.cursor()
        c.execute("Select stockID from stockMarket where stockSymbol==?",(stock_market,))
        stockID = c.fetchone()[0]
        c.execute("SELECT assetName,assetFullName FROM assets where stockID==? ORDER BY assetName ASC",(stockID,))
        stock = [{"assetName":name,"assetFullName":fullname} for name,fullname in c.fetchall()]
        c.fetchall()
        conn.close()
        return jsonify(stock)
        
    elif endpoint =="getStockMarkets_transaction":

        user_id = takeUserID(session['loginName'])
        transactionType = request.args.get("transactionType")
        conn = sqlite3.connect('DataBase1.db')
        c = conn.cursor()
        if transactionType == "Sell":
            c.execute("Select distinct stockSymbol FROM (Select sum(Quantity), userID, assetID, stockID from transactionHistory where userID==? and Quantity > 0 group by userID, assetID, stockID) a LEFT JOIN stockMarket b ON a.stockID = b.stockID",(user_id,))
        else:
            c.execute("Select stockSymbol from stockMarket")
        stockMarket = [row[0] for row in c.fetchall()]
        conn.close()
        return stockMarket

    elif endpoint == "getStockMarkets2":

        stock_market = request.args.get("stockMarket")
        user_id = takeUserID(session['loginName'])
        transactionType = request.args.get("transactionType")
        conn = sqlite3.connect('DataBase1.db')
        c = conn.cursor()
        if transactionType == "Sell":
            c.execute("SELECT stockSymbol FROM (SELECT SUM(quantity) AS quantity,userID,stockSymbol FROM(Select a.*,b.* FROM transactionHistory a LEFT JOIN stockMarket b ON a.stockID = b.stockID WHERE stockSymbol ==?)WHERE userID ==? group by userID, stockSymbol) WHERE quantity > 0)"
            ,(stock_market,user_id,))
        else:
            c.execute("Select stockID from stockMarket where stockSymbol==?",(stock_market,))
            stockID = c.fetchone()[0]
            c.execute("SELECT assetName, assetFullName FROM assets where stockID==? ORDER BY assetName ASC",(stockID,))
        stock = [{"assetName":name,"assetFullName":fullname} for name,fullname in c.fetchall()]
        c.fetchall()
        conn.close()
        return jsonify(stock)

    
    elif endpoint == "getInfo":

        stock = request.args.get('data')
        st = yf.Ticker(stock)
        return json.dumps(st.info)

    else:
        return "Bad endpoint", 400


# Return the JSON data for the Plotly graph
def gm(stock,period, interval):
    st = yf.Ticker(stock)
  
    # Create a line graph
    df = st.history(period=(period), interval=interval)
    df=df.reset_index()
    df.columns = ['Date-Time']+list(df.columns[1:])
    max = (df['Open'].max())
    min = (df['Open'].min())
    range = max - min
    margin = range * 0.05
    max = max + margin
    min = min - margin
    fig = px.area(df, x='Date-Time', y="Open",
        hover_data=("Open","Close","Volume"), 
        range_y=(min,max), template="seaborn" )

    # Create a JSON representation of the graph
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    return graphJSON

@app.route('/changePassword')
@login_required
def changePassword():
    title = 'Change Password'
    return render_template('changePassword.html', title=title)

@app.route('/changePasswordFun', methods=['POST','GET'])
def changePasswordFun():
    password = request.form.get('password', default=False)
    new_password = request.form.get('new_password1', default=False)

    loginName = session['loginName']
    userID = session['userID']

    if CheckIfCorrectCredentials(loginName, password) == True:
        changePasswordDB(userID, new_password)
        title = 'changePassword'
        return render_template('changePassword.html', success = True, title=title)
    else:
        title = 'changePassword'
        return render_template('changePassword.html', error = True, title=title)
    

def changePasswordDB(userID, new_password):
    conn = sqlite3.connect('DataBase1.db')
    c = conn.cursor()


    c.execute("update users set password = ? where userID = ?", (new_password, userID))

    conn.commit()
    conn.close()



@app.route('/portfolioOverview')
@login_required
def portfolioOverview():

    conn = sqlite3.connect('DataBase1.db')
    c = conn.cursor()
    userID = session['userID']
    c.execute(  "SELECT a.Sum_Quantity as Sum_Quantity, b.assetFullName, b.category, c.stockSymbol, c.openingTime, c.closingTime \
                FROM (SELECT userID, assetID, stockID, SUM(quantity) as Sum_Quantity \
                FROM transactionHistory where userID ==? group by userID,assetID,stockID) a \
                LEFT JOIN assets b ON a.assetID = b.assetID \
                LEFT JOIN stockMarket c ON a.stockID = c.stockID ORDER BY Sum_Quantity DESC", (userID, ))
    portfolio_data = c.fetchall()
    conn.close()
    title = 'Portfolio Overview'
    return render_template('portfolioOverview.html',title = title, portfolio_data=portfolio_data)


@app.route('/logout')
def logout():
    session.pop('loginName', None)
    session.pop('userID', None)
    return redirect('/index')


app.run(host='localhost', port=63700, use_reloader=False, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:63700
Press CTRL+C to quit
127.0.0.1 - - [23/Jan/2023 23:50:44] "GET /personalData HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2023 23:50:44] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [23/Jan/2023 23:50:44] "GET /static/script.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Jan/2023 23:50:45] "GET /changePassword HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2023 23:50:45] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [23/Jan/2023 23:50:45] "GET /script.js HTTP/1.1" 404 -
127.0.0.1 - - [23/Jan/2023 23:50:45] "GET /static/script.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Jan/2023 23:50:52] "POST /changePasswordFun HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2023 23:50:52] "


Present all data in student table:


GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [23/Jan/2023 23:50:52] "GET /script.js HTTP/1.1" 404 -
127.0.0.1 - - [23/Jan/2023 23:50:52] "GET /static/script.js HTTP/1.1" 304 -



Present all data in student table:


127.0.0.1 - - [23/Jan/2023 23:51:09] "POST /changePasswordFun HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2023 23:51:09] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [23/Jan/2023 23:51:09] "GET /script.js HTTP/1.1" 404 -
127.0.0.1 - - [23/Jan/2023 23:51:09] "GET /static/script.js HTTP/1.1" 304 -
